# Carregar dades

In [19]:
import pandas as pd
import pickle as pkl
import os
import librosa
import math

In [ ]:
#!pip install transformers==4.11.3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 907.8 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 4.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.9 MB/s eta 0:00:0000:01
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cp

In [20]:
data_path = os.path.join(os.getcwd(), "data")
db_full = pd.read_csv(os.path.join(data_path,"full_db.csv"))
db_full

,file,language,transcription
0,acoruna_barco_m67780_a84524_audio_converted.mp3,Galician,"En esta zona, tenemos 3 placas de discapacita..."
1,barcelona_Ovvero_m68255_a84999_audio_converted...,Bosnian,"Ate, da ste gravati. Ja, da ne? Ja, svoj mojo..."
2,berlin_Dirk_m67384_a84128_audio_converted.mp3,German,Also hier gibt es eine Kreuzung und neben den...
3,cuenca_Amalia_m69879_a86623_audio_converted.mp3,Spanish,"Asegún más caminando por la calle, se encuent..."
4,elbarcelonC3A8s_Martagosa_m71623_a88367_audio_...,Catalan,"Hola, bon dia, estem a les portes de l'estaci..."


In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

KeyboardInterrupt: 

In [22]:
lang_id_dict = {}
with open(os.path.join(os.getcwd(),'languages.txt'),'r') as f:
    for line in f:
        lang, lang_id = line.split(' \t')[0].strip(), line.split(' \t')[1].strip()
        lang_id_dict[lang] = lang_id
lang_id_dict

{'English': 'eng_Latn', 'Spanish': 'spa_Latn', 'Catalan': 'cat_Latn'}

In [23]:
lang_id_dict.keys()

dict_keys(['English', 'Spanish', 'Catalan'])

In [24]:
dbs = []
results2 = {}
i = 0
for lang in db_full['language'].unique():
    b = 0
    db = db_full[db_full['language'] == lang]
    text = list(db['transcription'])
    print(f'iteration {i}', lang, text)

    i += 1
    for j in text:
        if b > 0:
            continue
        try:
            math.isnan(j)
            results2[lang] = ['language_not_available' for _ in range(len(db))]
            b += 1
        except TypeError:
            pass
    
    if b > 0:
        continue

    try:
        src_lang = lang_id_dict[list(db['language'])[0]]
    
    except KeyError:
        results2[lang] = ['language_not_available' for _ in range(len(db))]
        continue

    translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang='eng_Latn', max_length = 400)
    results2[lang] = translator(text)
    
    with open(data_path+'/results_trans4.pkl','wb') as f:
        pkl.dump(results2,f)

    with open(data_path+'/results_trans5.pkl','wb') as f:
        pkl.dump(results2,f)

Device set to use cpu


iteration 0 Galician [' En esta zona, tenemos 3 placas de discapacitados.']
iteration 1 Bosnian [' Ate, da ste gravati. Ja, da ne? Ja, svoj mojos pozar selaci, no, svoj mojos, srja, vadla, amalisaj.']
iteration 2 German [' Also hier gibt es eine Kreuzung und neben den Ampel gibt es Taktideflasterung auf dem Boden, aber die Ampel selber gibt kein akutztisches Signal. Also es ist gefährlich hier rüber zu gehen alleine.']
iteration 3 Spanish [' Asegún más caminando por la calle, se encuentras con una valla de plástico bastante rígida porque se encuentran obras la casa de la derecha. Entonces, ¿qué es que es que es que va a la con mucho cuidado? Porque sólo queda como un metro, hasta que se termine la acera y ya entrese en la carretera.']


Device set to use cpu


iteration 4 Catalan [" Hola, bon dia, estem a les portes de l'estació del nord. Aquí està en música que crec que provés del tabé us de l'estació del nord, peles quals després també surt la màgapunia. Ara quan sortir la màgapunia farem una altra gravació. I quan sé, passa la moto també."]


In [26]:
results2

{'Galician': ['language_not_available'],
 'Bosnian': ['language_not_available'],
 'German': ['language_not_available'],
 'Spanish': [{'translation_text': "And then you walk down the street, and you find a pretty stiff plastic fence, because you're building the house on the right. So what's that that you're going to be very careful about?"}],
 'Catalan': [{'translation_text': "Hello, good morning, we're at the gates of the northern station. Here's music that I think comes from the taboo you of the northern station, which then also comes out the megapunia. Now when the megapunia comes out we'll do another recording. And when I know, the motorcycle goes too."}]}

In [60]:
dbs = []
known_langs = set()
known_langs.add('Italian')
i = 0
for lang in db_full['language'].unique():
    if lang in known_langs:
        continue
    if lang  == 'Chinese':
        break

    translations = []

    db1 = db_full[db_full['language'] == lang]
    print(i)
    for translation in results1[lang]:
        print(translation)
        if translation == 'language_not_available':
            translations.append(translation)
        else:
            translations.append(translation['translation_text'])

    known_langs.add(lang)

    dbs.append(db1.assign(translation=translations))

dbs[0]

0
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_not_available
language_

,Unnamed: 0.1,Unnamed: 0,file,language,transcription,translation
1,1,2,guatemalacity_ivan072_m67767_a84511_audio_conv...,Latin,So,language_not_available
46,46,49,municipalityofvalencia_Pamacas_m70393_a87137_a...,Latin,"Calma, me siento. Una brisa suave, recorre mi...",language_not_available
50,50,53,roma_Rita_m66276_a83028_audio_converted.mp3,Latin,hola,language_not_available
52,52,55,metropolitancityofvenice_PavelBerdon_m70098_a8...,Latin,S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S...,language_not_available
56,56,60,metropolitancityofvenice_RosaVicenti_m69411_a8...,Latin,Now we are in a really narrow street but I do...,language_not_available
...,...,...,...,...,...,...
4101,4101,4393,metropolitancityofvenice_Chiaravero_m69142_a85...,Latin,"I mean, Kale, the Lebo-Tegge and in front of ...",language_not_available
4107,4107,4399,metropolitancityofvenice_Ivisentin_m69342_a860...,Latin,M.S.,language_not_available
4124,4124,4418,metropolitancityofvenice_Ovvero_m68786_a85530_...,Latin,✌🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻...,language_not_available
4165,4165,4465,berlin_absonart_m67458_a84202_audio_converted.mp3,Latin,A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A...,language_not_available


In [62]:
i = 0
for lang in db_full['language'].unique():
    if lang in known_langs:
        continue

    translations = []

    db1 = db_full[db_full['language'] == lang]
    print(i)
    for translation in results2[lang]:
        print(translation)
        if translation == 'language_not_available':
            translations.append(translation)
        else:
            translations.append(translation['translation_text'])

    known_langs.add(lang)

    dbs.append(db1.assign(translation=translations))

dbs[0]

0
language_not_available
language_not_available
language_not_available
language_not_available
0
{'translation_text': 'This is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello that is hello 

,Unnamed: 0.1,Unnamed: 0,file,language,transcription,translation
1,1,2,guatemalacity_ivan072_m67767_a84511_audio_conv...,Latin,So,language_not_available
46,46,49,municipalityofvalencia_Pamacas_m70393_a87137_a...,Latin,"Calma, me siento. Una brisa suave, recorre mi...",language_not_available
50,50,53,roma_Rita_m66276_a83028_audio_converted.mp3,Latin,hola,language_not_available
52,52,55,metropolitancityofvenice_PavelBerdon_m70098_a8...,Latin,S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S...,language_not_available
56,56,60,metropolitancityofvenice_RosaVicenti_m69411_a8...,Latin,Now we are in a really narrow street but I do...,language_not_available
...,...,...,...,...,...,...
4101,4101,4393,metropolitancityofvenice_Chiaravero_m69142_a85...,Latin,"I mean, Kale, the Lebo-Tegge and in front of ...",language_not_available
4107,4107,4399,metropolitancityofvenice_Ivisentin_m69342_a860...,Latin,M.S.,language_not_available
4124,4124,4418,metropolitancityofvenice_Ovvero_m68786_a85530_...,Latin,✌🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻🏻...,language_not_available
4165,4165,4465,berlin_absonart_m67458_a84202_audio_converted.mp3,Latin,A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A-A...,language_not_available


In [64]:
dbs.append(db)

In [68]:
dbs[1]

,Unnamed: 0.1,Unnamed: 0,file,language,transcription,translation
2,2,3,santiagodecompostela_Carmenvillalobos_m67573_a...,Spanish,"Hola, seguimos aquí la ciudad de la cultura e...","Hey, we're still here, culture city. We're nex..."
35,35,37,santiagodecompostela_ClaraCordeiro_m67533_a842...,Spanish,de la tarna piscina ahora,From the pool now
93,93,99,municipalityofvalencia_Natalia28_m70327_a87071...,Spanish,Bueno hemos llegado a las razinas muy bolivas...,Well we've got to the very bolivian roots and ...
106,106,113,municipalityofvalencia_AnaSoler_m70402_a87146_...,Spanish,"Estamos delante de una fuente, atestamos toca...","We're in front of a fountain, we're touching w..."
130,130,139,santiagodecompostela_Carmenvillalobos_m67645_a...,Spanish,Estamos en el Instituto Shellmires.,We're at the Shellmires Institute.
...,...,...,...,...,...,...
4370,4370,4704,municipalityofvalencia_Natalia28_m70223_a86967...,Spanish,Es cubierto algo que nunca había visto en una...,It's covered in something I've never seen in a...
4382,4382,4727,municipalityofvalencia_Mari_m70351_a87095_audi...,Spanish,"Me tiene que repetir la agravación. Bueno, ya...","I have to repeat the aggravating. Well, it's n..."
4393,4393,4741,acoruna_Carmenvillalobos_m67833_a84577_audio_c...,Spanish,"Esto es Porto Simo, a 50 kilómetros aproximad...","This is Porto Simo, about 50 kilometers from S..."
4397,4397,4748,municipalityofvalencia_JustinaPerezCantos_m704...,Spanish,Estamos ya fuera del puerto. ¿Salgo bastante?...,"We're out of port, I'm getting out pretty good..."


In [71]:
len(dbs)

91

In [82]:
merged = dbs[0]
i = 0
for db in dbs:
    i += 1
    if i == 1:
        continue
    merged = pd.concat([merged, db], ignore_index=True)
merged

,Unnamed: 0.1,Unnamed: 0,file,language,transcription,translation
0,1,2,guatemalacity_ivan072_m67767_a84511_audio_conv...,Latin,So,language_not_available
1,46,49,municipalityofvalencia_Pamacas_m70393_a87137_a...,Latin,"Calma, me siento. Una brisa suave, recorre mi...",language_not_available
2,50,53,roma_Rita_m66276_a83028_audio_converted.mp3,Latin,hola,language_not_available
3,52,55,metropolitancityofvenice_PavelBerdon_m70098_a8...,Latin,S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S.S...,language_not_available
4,56,60,metropolitancityofvenice_RosaVicenti_m69411_a8...,Latin,Now we are in a really narrow street but I do...,language_not_available
...,...,...,...,...,...,...
4436,4301,4609,metropolitancityofvenice_Camira_m69415_a86159_...,Italian,Potrebbe sembrare un labirinto probabilmente ...,"It may seem like a maze, probably something li..."
4437,4304,4612,roma_Rita_m66423_a83175_audio_converted.mp3,Italian,arribando a questo in croccio molto difficile...,Arriving at this crossroads very difficult to ...
4438,4309,4617,metropolitancityofvenice_Davnera_m69558_a86302...,Italian,L'interno del spedale Santigione Paolo c'è un...,Inside the hospital Santigione Paolo there is ...
4439,4313,4622,roma_Viola_m66372_a83124_audio_converted.mp3,Italian,"Blinduiti, localizzare, registra in questa po...","Armor, locate, record in this button position."


In [ ]:
merged  = merged.sample(frac=1)
merged = merged.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
merged.to_csv(data_path+'/full_db1.csv')